## Music Generation V1


This script builds a model to learn music sequences from MIDI files. The model is a stacked LSTM model which takes as input a set of temporal notes and predicts the next note in the sequence

In [12]:
import os, random
import numpy as np
import pandas as pd
import glob

In [13]:
# Libs for music processing 
import msgpack
import mido
from mido import MidiFile

In [14]:
# Using music 21

from music21 import midi
from music21 import converter, instrument, note, chord, stream

In [15]:
# For preprocessing and modeling
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils

## Fetching data from midi files

They are extracted as notes

In [31]:
filename = '../data/kalyani.mid'
e = None
def get_notes():
    global e
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("../data/kalyani.mid"):
        mf = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(mf)
            print(s2.parts)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = mf.flat.notes

        for element in notes_to_parse:
            e = element
            if isinstance(element, note.Note):
                notes.append(str(element.nameWithOctave))
                # print(element.nameWithOctave)
            elif isinstance(element, chord.Chord):
                # print("Parsing a chord")
                notes.append('.'.join(str(n) for n in element.normalOrder))


    return notes

In [32]:
notes = get_notes()
len(notes)

Parsing ../data/kalyani.mid
<music21.stream.iterator.StreamIterator for Score:0x1355589b0 @:0>


77

## Data Processing 

Read the data as a sequence in to seq out

In [33]:
# Encode the notes in file
unique_notes = sorted(set(notes))
vocab_size = len(unique_notes)

# generate forward encoding
noteName_encoding = dict(
    (noteName, num) for num, noteName in enumerate(unique_notes)) 

# generate reverse encoding
encoding_to_note = dict(
    (number, note) for number, note in enumerate(unique_notes))


# Set training params
input_sequence_length = 10
x__inputs = []
y__outputs = []

# read the note stream and append to training data
for i in range(len(notes) - input_sequence_length):
    tmp_input = notes[i: i+input_sequence_length]
    tmp_output = notes[i+ input_sequence_length]
    x__inputs.append(list([noteName_encoding[ip] for ip in tmp_input]))
    y__outputs.append(noteName_encoding[tmp_output])

# Reshape the data since LSTMs want a 3-D input and a 1-D output
x__inputs = np.reshape(x__inputs, (len(x__inputs), input_sequence_length, 1))

# Normalize the data 
x__inputs = x__inputs/float(vocab_size-1)
y__outputs = np_utils.to_categorical(y__outputs)
print("Input stream shape %s" %str(np.array(x__inputs).shape))
print("Output stream shape %s" %str(np.array(y__outputs).shape))

Input stream shape (67, 10, 1)
Output stream shape (67, 20)


In [96]:
# Sanity check the data transform
assert(np.min(x__inputs) == 0)
assert(np.max(x__inputs) == 1)
assert(vocab_size == y__outputs.shape[1])
noteName_encoding.items()

dict_items([('A4', 0), ('A5', 1), ('B-4', 2), ('B3', 3), ('B4', 4), ('C#5', 5), ('C5', 6), ('D4', 7), ('D5', 8), ('E-4', 9), ('E-5', 10), ('E4', 11), ('E5', 12), ('F#4', 13), ('F#5', 14), ('F5', 15), ('G#4', 16), ('G#5', 17), ('G4', 18), ('G5', 19)])

## Model Training 

In [155]:
# Libs for m1odeling 
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.models import Sequential

In [156]:
model = Sequential()
model.add(LSTM(30, 
               input_shape = (input_sequence_length, 1),
               return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(30, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(30))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))

In [157]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [217]:
model.fit(x__inputs, y__outputs
          , epochs=3000
          , batch_size=10)

Epoch 1/3000
67/67 [==============================] - 0s 4ms/step - loss: 1.1825
Epoch 2/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.1878
Epoch 3/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.2022
Epoch 4/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.2005
Epoch 5/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.0833
Epoch 6/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.2558
Epoch 7/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.1328
Epoch 8/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.1330
Epoch 9/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.1272
Epoch 10/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.2634
Epoch 11/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.1564
Epoch 12/3000
67/67 [==============================] - 0s 3ms/step - loss: 1.1471
Epoch 13/3000
67/67 [====

67/67 [==============================] - 0s 5ms/step - loss: 0.8934
Epoch 102/3000
67/67 [==============================] - 0s 4ms/step - loss: 1.0117
Epoch 103/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.9057
Epoch 104/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.9045
Epoch 105/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.8971
Epoch 106/3000
67/67 [==============================] - 0s 4ms/step - loss: 0.9037
Epoch 107/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.8731
Epoch 108/3000
67/67 [==============================] - 0s 4ms/step - loss: 0.9181
Epoch 109/3000
67/67 [==============================] - 0s 4ms/step - loss: 1.0034
Epoch 110/3000
67/67 [==============================] - 0s 6ms/step - loss: 0.7984
Epoch 111/3000
67/67 [==============================] - 0s 4ms/step - loss: 0.8500
Epoch 112/3000
67/67 [==============================] - 0s 4ms/step - loss: 0.9092
Epoch 113/3000
67/6

67/67 [==============================] - 0s 3ms/step - loss: 0.5882
Epoch 298/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.5735
Epoch 299/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.4529
Epoch 300/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.6129
Epoch 301/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.5369
Epoch 302/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.5541
Epoch 303/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.4906
Epoch 304/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.5205
Epoch 305/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.5588
Epoch 306/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.4990
Epoch 307/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.5213
Epoch 308/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.5448
Epoch 309/3000
67/6

67/67 [==============================] - 0s 3ms/step - loss: 0.3437
Epoch 494/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.3541
Epoch 495/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.4186
Epoch 496/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.3689
Epoch 497/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.4547
Epoch 498/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.3241
Epoch 499/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.4411
Epoch 500/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.5489
Epoch 501/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.3557
Epoch 502/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.2959
Epoch 503/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.4227
Epoch 504/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.2984
Epoch 505/3000
67/6

67/67 [==============================] - 0s 3ms/step - loss: 0.2442
Epoch 690/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.2206
Epoch 691/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.1970
Epoch 692/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.2957
Epoch 693/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.2571
Epoch 694/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.2573
Epoch 695/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.2768
Epoch 696/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.1784
Epoch 697/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.2590
Epoch 698/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.2273
Epoch 699/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.2011
Epoch 700/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.5889
Epoch 701/3000
67/6

...

67/67 [==============================] - 0s 3ms/step - loss: 0.0331
Epoch 2051/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0498
Epoch 2052/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0595
Epoch 2053/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0124
Epoch 2054/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0341
Epoch 2055/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0851
Epoch 2056/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.1362
Epoch 2057/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.1022
Epoch 2058/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0905
Epoch 2059/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0068
Epoch 2060/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0044
Epoch 2061/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.1275
Epoch 20

67/67 [==============================] - 0s 3ms/step - loss: 0.0043
Epoch 2245/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0057
Epoch 2246/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0073
Epoch 2247/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0072
Epoch 2248/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0165
Epoch 2249/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.4547
Epoch 2250/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0266
Epoch 2251/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0267
Epoch 2252/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0392
Epoch 2253/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0379
Epoch 2254/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0037
Epoch 2255/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0393
Epoch 22

67/67 [==============================] - 0s 3ms/step - loss: 0.0418
Epoch 2439/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0834
Epoch 2440/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.1027
Epoch 2441/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0156
Epoch 2442/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0038
Epoch 2443/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0044
Epoch 2444/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0510
Epoch 2445/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.1094
Epoch 2446/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0177
Epoch 2447/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0625
Epoch 2448/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0108
Epoch 2449/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.1098
Epoch 24

67/67 [==============================] - 0s 3ms/step - loss: 0.0422
Epoch 2633/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0642
Epoch 2634/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0857
Epoch 2635/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0019
Epoch 2636/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0200
Epoch 2637/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.1600
Epoch 2638/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0360
Epoch 2639/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0288
Epoch 2640/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0048
Epoch 2641/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0057
Epoch 2642/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0019
Epoch 2643/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0051
Epoch 26

67/67 [==============================] - 0s 3ms/step - loss: 0.0082
Epoch 2827/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0068
Epoch 2828/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0502
Epoch 2829/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0254
Epoch 2830/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0115
Epoch 2831/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 2832/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0057
Epoch 2833/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0047
Epoch 2834/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0300
Epoch 2835/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.1494
Epoch 2836/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0067
Epoch 2837/3000
67/67 [==============================] - 0s 3ms/step - loss: 0.0079
Epoch 28

In [249]:
# Set seed note sequence
debug=False
start = int(random.randint(0, len(notes)))
pattern = notes[start: start+input_sequence_length]
ouput_notes = []
for i in range(10):
    # Encode and transform the note sequence
    encoded_note = [noteName_encoding[pattern_note] for pattern_note in pattern]
    transformed_note = np.array(encoded_note)/float(vocab_size-1)
    transformed_note = np.reshape(transformed_note, (1, input_sequence_length, 1))
    
    # Score the model
    preds = model.predict_classes(transformed_note)
    pred_note = encoding_to_note[preds[0]]
    ouput_notes.append(pred_note)
    if(debug):
        print('Iter %d | notes %s' %(i, pattern))
        print('pred_note is %s' %pred_note)
    
    # Update the scoring input pattern
    pattern.append(pred_note)
    pattern = pattern[-input_sequence_length: len(pattern)]

In [250]:
print("Predicted notes %s"%(ouput_notes))

Predicted notes ['G#4', 'C#5', 'B4', 'A4', 'A4', 'G4', 'B-4', 'C#5', 'C#5', 'D5']


In [254]:
# Save the music!
predicted_notes = [instrument.BaritoneSaxophone()]
offset = 0

for output_note in ouput_notes:
    print('Adding %s' %(output_note))
    n = note.Note(output_note, quarterLength=2.0)
    n.offset = offset
#     n.storedInstrument = instrument.PanFlute()
    predicted_notes.append(n)
    offset += 1.3
   
pred_stream = stream.Stream(predicted_notes)
pred_stream.write('midi', fp='../arches/data/test_output.mid')

Adding G#4
Adding C#5
Adding B4
Adding A4
Adding A4
Adding G4
Adding B-4
Adding C#5
Adding C#5
Adding D5


'../arches/data/test_output.mid'

In [255]:
pred_stream.elements

(<music21.instrument.BaritoneSaxophone Baritone Saxophone>,
 <music21.note.Note G#>,
 <music21.note.Note C#>,
 <music21.note.Note B>,
 <music21.note.Note A>,
 <music21.note.Note A>,
 <music21.note.Note G>,
 <music21.note.Note B->,
 <music21.note.Note C#>,
 <music21.note.Note C#>,
 <music21.note.Note D>)

In [256]:
n = note.Note(output_note)
n.__dict__

{'_activeSite': None,
 '_naiveOffset': 0.0,
 '_activeSiteStoredOffset': None,
 '_derivation': None,
 '_style': None,
 '_editorial': None,
 '_duration': <music21.duration.Duration 1.0>,
 '_priority': 0,
 'id': 5484651688,
 'groups': [],
 'sites': <music21.sites.Sites at 0x1469cd788>,
 'lyrics': [],
 'expressions': [],
 'articulations': [],
 'tie': None,
 '_notehead': 'normal',
 '_noteheadFill': None,
 '_noteheadParenthesis': False,
 '_stemDirection': 'unspecified',
 '_volume': None,
 'linkage': 'tie',
 'beams': <music21.beam.Beams >,
 'pitch': <music21.pitch.Pitch D5>}

In [246]:
n.pitch.__dir__()

['_groups',
 '_step',
 '_overridden_freq440',
 '_accidental',
 '_microtone',
 'defaultOctave',
 '_octave',
 'spellingIsInferred',
 'fundamental',
 '__module__',
 '__doc__',
 '_DOC_ORDER',
 '_twelfth_root_of_two',
 'classes',
 '_DOC_ATTR',
 '__init__',
 '__repr__',
 '__str__',
 '__eq__',
 '__deepcopy__',
 '__hash__',
 '__ne__',
 '__lt__',
 '__le__',
 '__gt__',
 '__ge__',
 'classSet',
 '_getGroups',
 '_setGroups',
 'groups',
 '_getAccidental',
 '_setAccidental',
 'accidental',
 '_getMicrotone',
 '_setMicrotone',
 'microtone',
 'isTwelveTone',
 'getCentShiftFromMidi',
 'alter',
 'convertQuarterTonesToMicrotones',
 'convertMicrotonesToQuarterTones',
 '_getPs',
 '_setPs',
 'ps',
 '_getMidi',
 '_setMidi',
 'midi',
 '_getName',
 '_setName',
 'name',
 'unicodeName',
 '_getNameWithOctave',
 '_setNameWithOctave',
 'nameWithOctave',
 'unicodeNameWithOctave',
 'fullName',
 '_getStep',
 '_setStep',
 'step',
 '_getPitchClass',
 '_setPitchClass',
 'pitchClass',
 '_getPitchClassString',
 'pitchClassSt